In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.models import Model

# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Define data directories
train_dir = "/content/drive/MyDrive/DIP/Train"
test_dir = "/content/drive/MyDrive/DIP/Test"

# Image dimensions
img_height, img_width = 224, 224
batch_size = 32

# Data augmentation and preprocessing
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

test_datagen = ImageDataGenerator(rescale=1./255)

# Load and augment training data
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical'
)

# Load test data
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical'
)

# Load pre-trained InceptionV3 model
base_model = InceptionV3(input_shape=(img_height, img_width, 3), include_top=False, weights='imagenet')

# Add custom classification head
x = GlobalAveragePooling2D()(base_model.output)
x = Dense(512, activation='relu')(x)
output = Dense(train_generator.num_classes, activation='softmax')(x)  # Adjusted to dynamically get number of classes

# Create model
model = Model(inputs=base_model.input, outputs=output)

# Freeze base layers
for layer in base_model.layers:
    layer.trainable = False

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(train_generator, epochs=10, validation_data=test_generator)

# Evaluate the model
test_loss, test_acc = model.evaluate(test_generator)
print("Test Accuracy:", test_acc)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Found 1280 images belonging to 4 classes.
Found 160 images belonging to 4 classes.
87910968/87910968 [==============================] - 1s 0us/step
Epoch 1/10
40/40 [==============================] - 487s 12s/step - loss: 1.0584 - accuracy: 0.5938 - val_loss: 0.8609 - val_accuracy: 0.6750
Epoch 2/10
40/40 [==============================] - 181s 5s/step - loss: 0.5404 - accuracy: 0.7328 - val_loss: 0.5457 - val_accuracy: 0.6562
Epoch 3/10
40/40 [==============================] - 189s 5s/step - loss: 0.4402 - accuracy: 0.7609 - val_loss: 0.5113 - val_accuracy: 0.7250
Epoch 4/10
40/40 [==============================] - 179s 4s/step - loss: 0.4266 - accuracy: 0.7734 - val_loss: 0.5333 - val_accuracy: 0.6812
Epoch 5/10
40/40 [==============================] - 181s 5s/step - loss: 0.4278 - accuracy: 0.7727 - val_loss: 0.4914 - val_accuracy: 0.6938
Epoch 6/10
40/40 